# Movie Recommendation Engine

**Goal**
1. Understand what is a recommendation engine and what kind of data is needed to conduct the analysis

2. Understand three types of recommendation engine:\
    (1) Content-based recommendation system\
    (2) Collaborative-filtering\
    (3) Hybrid recommendation system
    
3. Leverage the above knowledge to build movie recommendation engines using different methods

**Data**

To build a recommendation system, the data(feedback data) to be used is either explict or implicit. An explicit data directly has user's rating information while implicit data contains user's using behavior and actions which can also provide enough information for us to build a recommendation system.

The data I'm using here is provided by Datacamp, which is an explicit dataset, having the information of user id, movie's titles and user's ratings. 

In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [72]:
ratings_df = pd.read_csv('user_ratings.csv')
print('Dataset Overview: ')
print(ratings_df.head(5))

print('Data Columns: ')
print(ratings_df.info())

print('# of Ratings and Genres:')
print(ratings_df[['rating', 'genres']].nunique())

Dataset Overview: 
   userId  movieId  rating   timestamp             title  \
0       1        1     4.0   964982703  Toy Story (1995)   
1       5        1     4.0   847434962  Toy Story (1995)   
2       7        1     4.5  1106635946  Toy Story (1995)   
3      15        1     2.5  1510577970  Toy Story (1995)   
4      17        1     4.5  1305696483  Toy Story (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1  Adventure|Animation|Children|Comedy|Fantasy  
2  Adventure|Animation|Children|Comedy|Fantasy  
3  Adventure|Animation|Children|Comedy|Fantasy  
4  Adventure|Animation|Children|Comedy|Fantasy  
Data Columns: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  floa

In [73]:
ratings_df['rating'].value_counts().sort_index()

0.5     1370
1.0     2811
1.5     1791
2.0     7551
2.5     5550
3.0    20047
3.5    13136
4.0    26818
4.5     8551
5.0    13211
Name: rating, dtype: int64

## Non-personalized Recommendations

In [74]:
# Count the occurence of each movie title
movie_popularity = ratings_df['title'].value_counts()

# Obtain the top five common movies
print(movie_popularity.head(5).index)

Index(['Forrest Gump (1994)', 'Shawshank Redemption, The (1994)',
       'Pulp Fiction (1994)', 'Silence of the Lambs, The (1991)',
       'Matrix, The (1999)'],
      dtype='object')


The initial analysis tells us that the movie 'Forrest Gump (1994)' is the most rated movie among the entire dataset. To get a more accurate recommendation, explicit data column 'rating' is used in the next step.

In [57]:
# Find the mean of the ratings of each title
avg_ratings_df = ratings_df[['title','rating']].groupby('title').mean()

# Sort the average ratings from highest to lowest by rating
avg_ratings_df.sort_values(by='rating', ascending=False).head(5)

,rating
title,
Gena the Crocodile (1969),5.0
True Stories (1986),5.0
Cosmic Scrat-tastrophe (2015),5.0
Love and Pigeons (1985),5.0
Red Sorghum (Hong gao liang) (1987),5.0


After finding the average rating in given to each title, the result was showing above. However, since not every movies were rated by the same amount of people, the result might be affected by the number of user who had rated on the movie, that is in other words, the data might be affected by skewness. Therefore, in the next step, the recommendaion is going to be improved by setting the thredshold and reducing the skewness of the data.

In [70]:
# Create a list of only movies appearing > 50 times in the dataset
movie_popularity = ratings_df['title'].value_counts()
popular_movies = movie_popularity[movie_popularity.values>50].index

# Filter the popular movies from original data
popular_movies_rakings = ratings_df[ratings_df['title'].isin(popular_movies)]

# Find the top 5 movies base on their average ratings
popular_movies_rakings[['title','rating']].groupby('title').mean().sort_values(by='rating', ascending=False).head(5)

,rating
title,
"Shawshank Redemption, The (1994)",4.429022
"Godfather, The (1972)",4.289062
Fight Club (1999),4.272936
Cool Hand Luke (1967),4.271930
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),4.268041
